In [89]:
from collections import defaultdict, deque

N = 11


def get_num_digits_to_possible_solutions(generator, max_num_digits):
    num_digits_to_possible_solutions = defaultdict(list)
    while True:
        possible_solution = next(generator)
        num_digits = len(str(possible_solution))
        if num_digits < 2:
            continue
        if num_digits > max_num_digits:
            break
        num_digits_to_possible_solutions[num_digits].append(possible_solution)
    return num_digits_to_possible_solutions


def get_numbers_with_digit_sum(target_sum):
    queue = deque([(0, 0)])  # each tuple contains (current_number, current_sum)

    while queue:
        current_number, current_sum = queue.popleft()

        # Start the digit range from 1 if current_number is 0 to avoid leading zeros
        start_digit = 1 if current_number == 0 else 0

        for digit in range(start_digit, 10):
            next_number = current_number * 10 + digit
            next_sum = current_sum + digit
            if next_sum > target_sum:
                break  # No further numbers need to be checked beyond this digit
            elif next_sum == target_sum:
                yield next_number
            else:
                queue.append((next_number, next_sum))


def get_fibonacci():
    a, b = 1, 1
    while True:
        yield a
        a, b = b, a + b


def get_squares():
    i = 1
    while True:
        yield i**2
        i += 1


def get_multiples_of_thirty_seven():
    i = 1
    while True:
        yield 37 * i
        i += 1

In [90]:
n_by_one_shadings = []


def generate_all_shadings(i, prev, accum):
    if i == N:
        n_by_one_shadings.append(accum)
        return
    if prev == 1:
        generate_all_shadings(i + 1, 0, accum + [0])
    else:
        generate_all_shadings(i + 1, 0, accum + [0])
        generate_all_shadings(i + 1, 1, accum + [1])


generate_all_shadings(0, 0, [])
print(len(n_by_one_shadings))

n_by_one_shadings = [(shading, N - sum(shading)) for shading in n_by_one_shadings]
print(n_by_one_shadings[100])

233
([0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0], 8)


In [91]:
import numpy as np


def refine_regions(shading, regions):
    new_regions = []
    one_indices = [i for i, x in enumerate(shading) if x == 1]
    for region in regions:
        stack = []
        for x in region:
            if x in one_indices:
                if stack:
                    new_regions.append(stack)
                stack = []
            else:
                stack.append(x)
        if stack:
            new_regions.append(stack)

    return new_regions


def get_possible_solutions(num_digits_to_possible_solutions, regions):
    possible_solutions = []
    for shading, num_gaps in n_by_one_shadings:
        for possible_solution in num_digits_to_possible_solutions[num_gaps]:
            digits = [int(x) for x in str(possible_solution)]
            completed_row = [-1 for _ in range(N)]
            digit_index = 0
            for i in range(N):
                if shading[i] == 0:
                    completed_row[i] = digits[digit_index]
                    digit_index += 1

            is_valid = True
            refined_regions = refine_regions(shading, regions)
            for region in refined_regions:
                if not all(
                    completed_row[x] == completed_row[region[0]] for x in region
                ):
                    is_valid = False
            for region1, region2 in zip(refined_regions, refined_regions[1:]):
                if completed_row[region1[0]] == completed_row[region2[0]]:
                    is_valid = False
            if is_valid:
                possible_solutions.append(np.array(completed_row).reshape(1, -1))
    return np.stack(possible_solutions)

In [92]:
num_digits_to_digit_sum_sevens = get_num_digits_to_possible_solutions(
    get_numbers_with_digit_sum(7), N
)
digit_sum_sevens_regions = [[0], [1, 2], [3, 4], [5, 6], [7], [8, 9, 10]]
digit_sum_sevens = get_possible_solutions(
    num_digits_to_digit_sum_sevens, regions=digit_sum_sevens_regions
)
print(digit_sum_sevens.shape)
print(digit_sum_sevens[-1])

num_digits_to_fibonacci = get_num_digits_to_possible_solutions(get_fibonacci(), N)
fibonacci_regions = [[0], [1], [2, 3], [4, 5], [6], [7], [8], [9], [10]]
fibonnacci = get_possible_solutions(num_digits_to_fibonacci, regions=fibonacci_regions)
print(fibonnacci.shape)

num_digits_to_squares = get_num_digits_to_possible_solutions(get_squares(), N)
square2_regions = [[0, 1, 2], [3, 4, 5], [6, 7], [8, 9, 10]]
squares2 = get_possible_solutions(num_digits_to_squares, regions=square2_regions)
print(squares2.shape)

# num_digits_to_multiples_of_thirty_seven = get_num_digits_to_possible_solutions(get_multiples_of_thirty_seven(), N)
# multiples_of_thirty_seven_regions = [[0], [1,2,3,4], [5,6], [7], [8,9,10]]
# thirty_sevens = get_possible_solutions(num_digits_to_multiples_of_thirty_seven, regions=multiples_of_thirty_seven_regions)
# print(thirty_sevens.shape)

(2498, 1, 11)
[[-1  5 -1  0 -1  1 -1  0 -1  1 -1]]
(235, 1, 11)
(7614, 1, 11)


In [99]:
print(digit_sum_sevens[0])

[[ 1  0  0  1  1  0  0  2  1  1 -1]]


In [94]:
def merge(tops, bottoms, C):
    top_last_row = tops[:, -1, :]
    bottom_first_row = bottoms[:, 0, :]
    top_last_row_expanded = top_last_row[:, np.newaxis, :]
    bottom_first_row_expanded = bottom_first_row[np.newaxis, :, :]

    # Compatibility check
    equality = np.equal(top_last_row_expanded, bottom_first_row_expanded)
    inequality = np.not_equal(top_last_row_expanded, bottom_first_row_expanded)

    # Mask from C for checking conditions
    equality_mask = (C == 1).T  # shape (1, G)
    inequality_mask = (C == 0).T  # shape (1, G)

    # Perform checks
    compatibility = np.where(equality_mask, equality, inequality)

    # Check for no adjacent -1s
    no_adjacent_minus_ones = ~(
        np.logical_and(top_last_row_expanded == -1, bottom_first_row_expanded == -1)
    )

    # Combine all conditions
    total_compatibility = np.logical_and(compatibility, no_adjacent_minus_ones)

    # Check across all G for final compatibility
    compatible_solutions_indices = np.where(np.all(total_compatibility, axis=2))

    # Gather valid solutions and concatenate
    valid_top = tops[compatible_solutions_indices[0]]
    valid_bottom = bottoms[compatible_solutions_indices[1]]

    # Final tensor with shape (N3, M1+M2, G)
    final_tensor = np.concatenate([valid_top, valid_bottom], axis=1)

    return final_tensor


merged_fibonacci_squares = merge(
    fibonnacci, squares2, np.array([1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0])
)

merged_digit_sum_sevens_fibonacci = merge(
    digit_sum_sevens,
    fibonnacci,
    np.array([1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1]),
)

merge_all_three = merge(
    digit_sum_sevens,
    merged_fibonacci_squares,
    np.array([1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1]),
)

In [98]:
print(merged_fibonacci_squares)

[[[ 3  9  0 -1  8  8  1  6 -1  9 -1]
  [ 3 -1  1  8  8  8 -1  6  0 -1  9]]

 [[ 3  9 -1  0  8  8  1  6 -1  9 -1]
  [ 3 -1  1  8  8  8 -1  6  0 -1  9]]]


In [96]:
print(merged_digit_sum_sevens_fibonacci)

[]
